In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch_ga import GeometricAlgebra
from torch_ga.layers import GeometricProductDense, TensorToGeometric, GeometricToTensor

In [ ]:
# !pip install matplotlib
# !pip3 install torch torchvision torchaudio
# !conda install -y pytorch torchvision -c pytorch
# !conda update -n base -c defaults conda
# !conda install pytorch_lightning -c conda-forge
# !pip install pytorch-lightning
# !conda remove -y pytorch torchvision  
# !conda install -y pytorch torchvision torchaudio -c pytorch-nightly  #for python 3.11
# !micromamba install -y pytorch torchvision -c pytorch
# !micromamba install -y matplotlib

In [ ]:
def make_batch(batch_size):
    # triangle_points = torch.rand([batch_size, 3, 2], minval=-1, maxval=1)
    triangle_points = torch.rand([batch_size, 3, 2])
    # triangle_points = torch.FloatTensor(shape=[batch_size, 3, 2])
    # print(f"triangle_points.shpae={triangle_points.shape}")
    triangle_points = triangle_points.uniform_(-1, 1)
    x, y = triangle_points[..., 0], triangle_points[..., 1]
    # print([batch_size, 3, 2])
    # print(f"triangle_points.shpae={triangle_points.shape}")
    # print(f"x.shpae={x.shape}")
    # print(f"y.shpae={y.shape}")
    ax, ay, bx, by, cx, cy = x[..., 0], y[..., 0], x[..., 1], y[..., 1], x[..., 2], y[..., 2]
    triangle_areas = 0.5 * torch.abs(ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
    return triangle_points, triangle_areas

num_samples = 4
sample_points, sample_areas = make_batch(num_samples)

fig, axes = plt.subplots(1, num_samples, figsize=(12, 4), sharex=True, sharey=True)
for i, ax in enumerate(axes):
    points = sample_points[i]
    area = sample_areas[i]
    # center = tf.reduce_mean(points, axis=0)
    center = points.mean(dim=0)
    ax.scatter(points[..., 0], points[..., 1])
    ax.add_patch(plt.Polygon(points))
    ax.annotate("Area: %.2f" % area, center)
fig.show()

In [ ]:
ga = GeometricAlgebra([1, 1])
s_indices = [0]
v_indices = [1, 2]
mv_indices = torch.arange(ga.num_blades)

class Squeeze(nn.Module):
    def __init_(self):
        super().__init__()
    def forward(self,x):
        return torch.squeeze(x)
    
model = nn.Sequential(
    TensorToGeometric(ga, blade_indices=v_indices),
    GeometricProductDense(
        ga, units=128, activation="relu",
        blade_indices_kernel=mv_indices,
        blade_indices_bias=mv_indices,
    ),
    # GeometricProductDense(
    #     ga, units=16, activation="relu",
    #     blade_indices_kernel=mv_indices,
    #     blade_indices_bias=mv_indices,
    # ),
    # GeometricProductDense(
    #     ga, units=16, activation="relu",
    #     blade_indices_kernel=mv_indices,
    #     blade_indices_bias=mv_indices,
    # ),
    GeometricProductDense(
        ga, units=1,
        blade_indices_kernel=mv_indices,
        blade_indices_bias=s_indices,
    ),
    GeometricToTensor(ga, blade_indices=s_indices),
    # nn.Flatten(),
    Squeeze(),
)

# [B, 3, 2]

print("Samples:", sample_points)
print("Model(Samples):", model(sample_points))

In [ ]:
# !pip install pytorch_lightning
# !pip install --upgrade pytorch_lightning
# !pip install --no-cache-dir pytorch_lightning
# !pip install rich
# !pip uninstall -y rich
# !pip uninstall -y pytorch_lightning
# !pip install --no-cache-dir pygments
# !pip uninstall -y pygments
# !pip install pygments==2.14
# !pip install pygments
# !pip install --upgrade rich

# !pip list | grep pygments

# !conda install -y -c conda-forge pytorch-lightning 

    # lightning-utilities-0.7.1  |   py38hecd8cb5_0          31 KB
    # pytorch-lightning-1.9.3    |     pyhd8ed1ab_0         433 KB  conda-forge
    # torchmetrics-0.11.2        |   py38h01d92e1_0         425 KB
    
    
# !conda install -y -c conda-forge rich

# !mamba install -y -c conda-forge pytorch-lightning 

 

In [ ]:
train_points, train_areas = make_batch(1024)
test_points, test_areas = make_batch(128)

import pytorch_lightning as pl
# import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

class Lighting(pl.LightningModule):
    def __init__(self,module):
        super().__init__()
        self.module = module
        self.loss = nn.MSELoss()        
    def forward(self, x):
        return self.module(x)
    # def configure_optimizers(self):
    #     # return torch.optim.Adam(self.module.parameters(), lr=0.02)
    #     return torch.optim.Adam(self.module.parameters(), lr=1e-1)
    def configure_optimizers(self):
        # optimizer = torch.optim.Adam(self.module.parameters(), lr=2e-2)
        optimizer = torch.optim.SGD(self.module.parameters(), lr=2e-2)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
        # ['train_loss', 'train_loss_step', 'val_loss', 'train_loss_epoch'] "metric_to_track"
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "train_loss"}
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self.module(x)
        loss = self.loss(y,y_hat)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.module(x)
        loss = self.loss(y_hat, y)
        self.log("val_loss", loss)        



# def compile(model, optimizer,loss,metrics,learning_rate=1e-3):
#     if optimizer in ['Adam']: 
#         optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     if loss in ['mse']: 
#         loss = nn.MSELoss()
#     return optimizer,loss

# def fit(model, x, y, validation_data, epochs, optimizer):
#     for ep in epochs:
#         for xi,yi in zip(x,y):
#             y_ = model(xi)
#             loss_ = loss(yi,y_)
            
        

# optimizer, loss = compile(model, optimizer="Adam", loss="mse", metrics=["mae"])
# model.fit(x=train_points, y=train_areas, validation_data=(test_points, test_areas), epochs=100)

net = Lighting(model)

# tr_dataloader = DataLoader(TensorDataset(train_points, train_areas), batch_size=128, shuffle=True, drop_last=True)
# va_dataloader = DataLoader(TensorDataset(test_points, test_areas), batch_size=128, shuffle=True, drop_last=True)

# tr_dataloader = DataLoader(TensorDataset(train_points, train_areas), batch_size=32, shuffle=True, drop_last=True)
# va_dataloader = DataLoader(TensorDataset(test_points, test_areas), batch_size=32, shuffle=False, drop_last=True)

tr_dataloader = DataLoader(TensorDataset(train_points, train_areas), batch_size=16, shuffle=True)
va_dataloader = DataLoader(TensorDataset(test_points, test_areas), batch_size=16, shuffle=False)



# from pytorch_lightning.callbacks import RichProgressBar

# trainer = Trainer(callbacks=[RichProgressBar()])
# trainer = pl.Trainer(max_epochs=100,progress_bar_refresh_rate=20,callbacks=[RichProgressBar()]) 
# trainer = pl.Trainer(max_epochs=10,progress_bar_refresh_rate=2,callbacks=[RichProgressBar()]) 
# gpus=1,

# trainer = pl.Trainer(max_epochs=10,progress_bar_refresh_rate=2) 

trainer = pl.Trainer(max_epochs=100)

trainer.fit(net, train_dataloaders=tr_dataloader, val_dataloaders = va_dataloader)



In [ ]:
print(model)

In [ ]:
ns = 10
test_points[:ns],test_areas[:ns], model(test_points[:ns])

In [ ]:
# # print(model.summary())

# https://www.pytorchlightning.ai/blog/using-optuna-to-optimize-pytorch-lightning-hyperparameters

# def objective(trial):
#     # PyTorch Lightning will try to restore model parameters from previous trials if checkpoint
#     # filenames match. Therefore, the filenames for each trial must be made unique.
#     checkpoint_callback = pl.callbacks.ModelCheckpoint(
#         os.path.join(MODEL_DIR, "trial_{}".format(trial.number)), monitor="accuracy"
#     )

#     # The default logger in PyTorch Lightning writes to event files to be consumed by
#     # TensorBoard. We create a simple logger instead that holds the log in memory so that the
#     # final accuracy can be obtained after optimization. When using the default logger, the
#     # final accuracy could be stored in an attribute of the `Trainer` instead.
#     logger = DictLogger(trial.number)

#     trainer = pl.Trainer(
#         logger=logger,
#         val_percent_check=PERCENT_TEST_EXAMPLES,
#         checkpoint_callback=checkpoint_callback,
#         max_epochs=EPOCHS,
#         gpus=0 if torch.cuda.is_available() else None,
#         early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="accuracy"),
#     )

#     model = LightningNet(trial)
#     trainer.fit(model)

#     return logger.metrics[-1]["accuracy"]

In [ ]:
predicted_sample_areas = model(sample_points)

fig, axes = plt.subplots(1, num_samples, figsize=(20, 5), sharex=True, sharey=True)
for i, ax in enumerate(axes):
    points = sample_points[i]
    area = sample_areas[i]
    predicted_area = predicted_sample_areas[i]
    center = points.mean(dim=0)
    ax.scatter(points[..., 0], points[..., 1])
    ax.add_patch(plt.Polygon(points))
    ax.annotate("Area: %.2f" % area, center)
    ax.annotate("Predicted : %.2f" % predicted_area, center + torch.tensor([0, -0.1]))
fig.show()

In [ ]:
predicted_sample_areas = model(sample_points)

fig, axes = plt.subplots(1, num_samples, figsize=(20, 5), sharex=True, sharey=True)
for i, ax in enumerate(axes):
    points = sample_points[i]
    area = sample_areas[i]
    predicted_area = predicted_sample_areas[i]
    center = torch.mean(points, dim=0)
    ax.scatter(points[..., 0], points[..., 1])
    ax.add_patch(plt.Polygon(points))
    ax.annotate("Area: %.2f" % area, center)
    ax.annotate("Predicted area: %.2f" % predicted_area, center + torch.tensor([0, -0.1]))
fig.show()

In [ ]:
model_normal = nn.Sequential(
    nn.Linear(6,64*2),
    nn.ReLU(),
    nn.Linear(64*2,64*2),
    nn.ReLU(),
    nn.Linear(64*2,1)
)


net = Lighting(model_normal)

tr_dataloader = DataLoader(TensorDataset(train_points.view(-1,6), train_areas), batch_size=128, shuffle=True, drop_last=True)
va_dataloader = DataLoader(TensorDataset(test_points.view(-1,6), test_areas), batch_size=128, shuffle=True, drop_last=True)

trainer = pl.Trainer(max_epochs=100)

trainer.fit(net, train_dataloaders=tr_dataloader, val_dataloaders = va_dataloader)


# model_normal.compile(optimizer="Adam", loss="mse", metrics=["mae"])
# model_normal.fit(x=tf.reshape(train_points, [-1, 6]), y=train_areas, validation_data=(tf.reshape(test_points, [-1, 6]), test_areas), epochs=100)

In [ ]:
print(model_normal)

In [ ]:
predicted_sample_areas = model_normal(sample_points.view(-1,6))

fig, axes = plt.subplots(1, num_samples, figsize=(20, 5), sharex=True, sharey=True)
for i, ax in enumerate(axes):
    points = sample_points[i]
    area = sample_areas[i]
    predicted_area = predicted_sample_areas[i]
    center = points.mean(dim=0)
    ax.scatter(points[..., 0], points[..., 1])
    ax.add_patch(plt.Polygon(points))
    ax.annotate("Area: %.2f" % area, center)
    ax.annotate("Predicted : %.2f" % predicted_area, center + torch.tensor([0, -0.1]))
fig.show()

In [ ]:
print(model_normal.summary())

In [ ]:
predicted_sample_areas = model_normal(torch.reshape(sample_points, [-1, 6]))

fig, axes = plt.subplots(1, num_samples, figsize=(20, 5), sharex=True, sharey=True)
for i, ax in enumerate(axes):
    points = sample_points[i]
    area = sample_areas[i]
    predicted_area = predicted_sample_areas[i]
    center = torch.mean(points, axis=0)
    ax.scatter(points[..., 0], points[..., 1])
    ax.add_patch(plt.Polygon(points))
    ax.annotate("Area: %.2f" % area, center)
    ax.annotate("Predicted area: %.2f" % predicted_area, center + torch.tensor([0, -0.1]))
fig.show()